<a href="https://colab.research.google.com/github/williambrunos/Deep-Learning-Neuro-evolution/blob/main/First_Implementations/first_implementations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Implementations with tensorflow

## Abstract

This notebook tries to implement some neural netowork used to solve some problem and tries to evolve thw weights of it using neural evolution, implemented with [tensorflow](https://www.tensorflow.org/probability/api_docs/python/tfp/optimizer/differential_evolution_minimize)

````Python
tfp.optimizer.differential_evolution_minimize(
    objective_function,
    initial_population=None,
    initial_position=None,
    population_size=50,
    population_stddev=1.0,
    max_iterations=100,
    func_tolerance=0,
    position_tolerance=1e-08,
    differential_weight=0.5,
    crossover_prob=0.9,
    seed=None,
    name=None
)
````

**obs**: if you read the documentation of this function, you'll see that the evolutionary algorithm will try to minimize the objective function, rather than maximize it. 

"A candidate s_1 is considered better than s_2 if f(s_1) < f(s_2)." - The function documentation

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [7]:
import tensorflow_probability as tfp

In [18]:
population_size = 40
# With an initial population and a multi-part state.
initial_population = (tf.random.normal([population_size]),
                        tf.random.normal([population_size]))
def easom_fn(x, y):
  return -(tf.math.cos(x) * tf.math.cos(y) *
             tf.math.exp(-(x-np.pi)**2 - (y-np.pi)**2))

optim_results = tfp.optimizer.differential_evolution_minimize(
    easom_fn,
    initial_population=initial_population,
    max_iterations=50,
    seed=43210)

print(optim_results.converged)
print(optim_results.position)  # Should be (close to) [pi, pi].
print(optim_results.objective_value)    # Should be -1.
print(optim_results.final_population) # final population
print(optim_results.objective_value) # final value of objective fun. on the final position
print(optim_results.final_objective_values)

# With a single starting point
initial_position = (tf.constant(1.0), tf.constant(1.0))

optim_results = tfp.optimizer.differential_evolution_minimize(
    easom_fn,
    initial_position=initial_position,
    population_size=40,
    population_stddev=2.0,
    seed=43210)

"""
A real Tensor of any shape. The seed solution used to initialize the population of solutions. If this parameter is specified then initial_population must not be specified.
"""

tf.Tensor(False, shape=(), dtype=bool)
[<tf.Tensor: shape=(), dtype=float32, numpy=1.3049982>, <tf.Tensor: shape=(), dtype=float32, numpy=1.3049858>]
tf.Tensor(-8.110225e-05, shape=(), dtype=float32)
[<tf.Tensor: shape=(40,), dtype=float32, numpy=
array([1.304923 , 1.3048488, 1.305006 , 1.3050697, 1.304935 , 1.3049966,
       1.3050647, 1.3049982, 1.3049098, 1.3049395, 1.305081 , 1.3048955,
       1.3050886, 1.3051233, 1.3048406, 1.3049976, 1.3049233, 1.3048828,
       1.3048644, 1.3049314, 1.3049678, 1.3050447, 1.3048974, 1.3050356,
       1.3048602, 1.305109 , 1.3051052, 1.3049057, 1.304926 , 1.305162 ,
       1.3049338, 1.3049052, 1.3049569, 1.3051031, 1.3049951, 1.304913 ,
       1.3050461, 1.3049512, 1.3050075, 1.3048885], dtype=float32)>, <tf.Tensor: shape=(40,), dtype=float32, numpy=
array([1.3050191, 1.3049475, 1.3050473, 1.3049636, 1.3050191, 1.304862 ,
       1.3050528, 1.3049858, 1.3049417, 1.3049017, 1.3049157, 1.3049911,
       1.3049556, 1.3049157, 1.3049328, 1.3049623, 1

'\nA real Tensor of any shape. The seed solution used to initialize the population of solutions. If this parameter is specified then initial_population must not be specified.\n'

In [11]:
np.array(optim_results.position)

array([3.1416764, 3.1414824], dtype=float32)